In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# 3-harmonic model definition
def harmonic_3(t, a0, a1, b1, a2, b2, a3, b3):
    T = len(t)
    return (
        a0
        + a1 * np.sin(2 * np.pi * 1 * t / T) + b1 * np.cos(2 * np.pi * 1 * t / T)
        + a2 * np.sin(2 * np.pi * 2 * t / T) + b2 * np.cos(2 * np.pi * 2 * t / T)
        + a3 * np.sin(2 * np.pi * 3 * t / T) + b3 * np.cos(2 * np.pi * 3 * t / T)
    )

# Load data
df = pd.read_csv("Karnataka_Datasets/Across/Clean/Bangalore_train_height_duration_structure_train.csv")

# Time vector from VV columns
vv_cols = [col for col in df.columns if col.startswith("VV_")]
vh_cols = [col for col in df.columns if col.startswith("VH_")]
t = np.arange(1, len(vv_cols) + 1)

# Metadata columns to retain
meta_cols = ['Crop_Name', 'Height', 'height_numeric', 'Structure',
             'Structure_Numeric', 'Duration', 'duration_numeric']

# Lists to hold results
results = []

# Iterate row-wise and fit harmonics
for idx, row in df.iterrows():
    try:
        vv_vals = row[vv_cols].astype(float).values
        vh_vals = row[vh_cols].astype(float).values

        vv_params, _ = curve_fit(harmonic_3, t, vv_vals, maxfev=10000)
        vh_params, _ = curve_fit(harmonic_3, t, vh_vals, maxfev=10000)

    except Exception as e:
        print(f"Row {idx} failed: {e}")
        vv_params = [np.nan] * 7
        vh_params = [np.nan] * 7

    # Combine metadata + fitted coefficients
    combined_row = list(row[meta_cols]) + list(vv_params) + list(vh_params)
    results.append(combined_row)

# Column names for the result
vv_coef_cols = [f'VV_{p}' for p in ['a0', 'a1', 'b1', 'a2', 'b2', 'a3', 'b3']]
vh_coef_cols = [f'VH_{p}' for p in ['a0', 'a1', 'b1', 'a2', 'b2', 'a3', 'b3']]
result_cols = meta_cols + vv_coef_cols + vh_coef_cols

# Create final DataFrame
result_df = pd.DataFrame(results, columns=result_cols)

# Save to CSV
result_df.to_csv("Karnataka_Datasets/Across/Clean/Bangalore_train_height_duration_structure_train_harmonic.csv", index=False)
